# Natural Language Processing Code Along Project

### 1. Import koniecznych pakietów i wstępna obróbka danych

Zaczynamy od importowania pakietu findspark, aby w dalszej kolejności umożliwić import Sparka i jego klas.

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

Import danych z pliku CSV:

In [3]:
data = spark.read.csv('SMSSpamCollection', inferSchema=True, sep='\t')

In [5]:
data.columns

['_c0', '_c1']

Zmieńmy nazwę kolumn nna bardziej intuicyjne:

In [6]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [7]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [8]:
from pyspark.sql.functions import length

In [9]:
data = data.withColumn('length', length(data['text']))

Sprawdźmy jaką długość mają poszczególne wiadomości:

In [10]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



Nastepnie sprawdźmy jaką średnią długośc mają wiadomości w obszarze pojedyńczej klasy:

In [11]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [12]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [13]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')

In [14]:
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')

In [15]:
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')

In [16]:
idf = IDF(inputCol='c_vec', outputCol='tf_idf')

Zamienia 'labal' z typu string na wartości numeryczne:

In [17]:
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

Używane biblioteki wymagają aby wszystkie interesujące nas featury umieścić wewnątrz list.

In [18]:
from pyspark.ml.feature import VectorAssembler

In [19]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features' )

In [22]:
from pyspark.ml import Pipeline

In [23]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [25]:
cleaner = data_prep_pipe.fit(data)

In [26]:
clean_data = cleaner.transform(data)

In [27]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [29]:
clean_data = clean_data.select('label', 'features')

In [30]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13459,[8,12,33,6...|
|  0.0|(13459,[0,26,308,...|
|  1.0|(13459,[2,14,20,3...|
|  0.0|(13459,[0,73,84,1...|
|  0.0|(13459,[36,39,140...|
|  1.0|(13459,[11,57,62,...|
|  0.0|(13459,[11,55,108...|
|  0.0|(13459,[133,195,4...|
|  1.0|(13459,[1,50,124,...|
|  1.0|(13459,[0,1,14,29...|
|  0.0|(13459,[5,19,36,4...|
|  1.0|(13459,[9,18,40,9...|
|  1.0|(13459,[14,32,50,...|
|  0.0|(13459,[42,99,101...|
|  0.0|(13459,[567,1745,...|
|  1.0|(13459,[32,113,11...|
|  0.0|(13459,[86,224,47...|
|  0.0|(13459,[0,2,52,13...|
|  0.0|(13459,[0,77,107,...|
|  1.0|(13459,[4,32,35,6...|
+-----+--------------------+
only showing top 20 rows



Podział danych na zestaw treningowy i testowy:

In [34]:
training, test = clean_data.randomSplit([0.7, 0.3])

### 2. Klasyfikacja

Klasyfikacja odbędzie się przy użyciu modelu Bayesa:

In [35]:
from pyspark.ml.classification import NaiveBayes

In [36]:
nb = NaiveBayes()

In [37]:
spam_detector = nb.fit(training)

In [38]:
test_results = spam_detector.transform(test)

In [39]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13459,[0,1,2,14,...|[-609.78999121048...|[1.0,7.3116832988...|       0.0|
|  0.0|(13459,[0,1,5,15,...|[-745.01480486853...|[1.0,1.2163457737...|       0.0|
|  0.0|(13459,[0,1,6,16,...|[-1002.6097636545...|[1.0,3.9872451569...|       0.0|
|  0.0|(13459,[0,1,8,9,1...|[-878.57552873872...|[1.0,3.8286965129...|       0.0|
|  0.0|(13459,[0,1,8,9,1...|[-1156.5890249750...|[1.0,7.9966809051...|       0.0|
|  0.0|(13459,[0,1,8,16,...|[-658.33958159034...|[1.0,1.5398268333...|       0.0|
|  0.0|(13459,[0,1,10,15...|[-537.40508065451...|[1.0,5.9596320939...|       0.0|
|  0.0|(13459,[0,1,21,29...|[-973.17444321061...|[0.99999999999999...|       0.0|
|  0.0|(13459,[0,1,24,29...|[-1009.7425938083...|[1.0,2.9600724959...|       0.0|
|  0.0|(13459,[0

### 3. Ocena skuteczności modelu

In [41]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [42]:
acc_eval = MulticlassClassificationEvaluator()

In [44]:
acc = acc_eval.evaluate(test_results)

In [45]:
print('ACC of NB Model')
print(acc)

ACC of NB Model
0.9203630165278087
